In [1]:
import sys
import glob
import torch
import torch.nn.functional as F
import numpy as np
import pandas as ps
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn import metrics

import albumentations as alb
import albumentations.pytorch


sys.path.append("..")
device = torch.device("cuda:0")

In [2]:
from src.models import MulticlassEfficientNet
from src.datasets import ImagesDataset

normalizations = alb.Compose([alb.Normalize(), alb.pytorch.ToTensorV2()])
# normalizations = alb.Compose([alb.ToFloat(max_value=255), alb.pytorch.ToTensorV2()])
# normalizations = alb.Compose([alb.Normalize(max_pixel_value=1), alb.pytorch.ToTensorV2()])  # IT IS WRONG

ttas = [
    # original
    alb.Compose([
        alb.Resize(512, 512),
        normalizations,
    ]),
    # horizontal flipped
    alb.Compose([
        alb.Resize(512, 512),
        alb.HorizontalFlip(p=1),
        normalizations,
    ]),
    # vertical flipped
    alb.Compose([
        alb.Resize(512, 512),
        alb.VerticalFlip(p=1),
        normalizations,
    ]),
    # horizontal & vertical flipped
    alb.Compose([
        alb.Resize(512, 512),
        alb.HorizontalFlip(p=1),
        alb.VerticalFlip(p=1),
        normalizations,
    ]),
]

len(ttas)

/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/nbformat/notebooknode.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size

4

In [3]:
model = MulticlassEfficientNet("efficientnet-b0", 4)
model.load_state_dict(
    torch.load(
        "../logs/efficientnet-b0-c4-continue2/checkpoints/best.pth",
#         "../logs/efficientnet-b0-without-normalization/checkpoints/best.pth",
        map_location="cpu"
    )["model_state_dict"]
)
model = model.to(device)
model = model.eval()

Loaded pretrained weights for efficientnet-b0


In [4]:
test_images = glob.glob("../data/resized_data/Test/*.jpg")

len(test_images), test_images[:2]

(5000,
 ['../data/resized_data/Test/1787.jpg', '../data/resized_data/Test/0312.jpg'])

In [5]:
tta_preds = []
for transform_idx, transform in enumerate(ttas, 1):

    dataset = ImagesDataset(test_images, transforms=transform)
    loader = DataLoader(dataset, batch_size=64, num_workers=16)

    preds = []
    with torch.no_grad():
        for batch in tqdm(loader, desc=f"TTA index - {transform_idx}"):
            batch = batch.to(device)
            out = 1 - F.softmax(model(batch), dim=1).data.cpu().numpy()[:,0]
#             out = torch.sigmoid().detach().cpu().numpy().flatten()
            preds.append(out)

    preds = np.concatenate(preds)
    tta_preds.append(preds)

TTA index - 4: 100%|██████████| 79/79 [00:38<00:00,  2.08it/s]


In [6]:
avg_pred = np.mean(np.stack(tta_preds), 0)
avg_pred.shape, avg_pred

((5000,),
 array([0.5663949 , 0.97354424, 0.48302984, ..., 0.6806608 , 0.958826  ,
        0.66003793], dtype=float32))

In [7]:
submission = ps.DataFrame.from_dict({
    "Id": [file.rsplit("/", 1)[1] for file in test_images],
    "Label": avg_pred
}).sort_values(by="Id").reset_index(drop=True)

print(submission.shape)
submission.head()

(5000, 2)


,Id,Label
0,0001.jpg,0.094490
1,0002.jpg,0.637682
2,0003.jpg,0.591151
3,0004.jpg,0.625014
4,0005.jpg,0.824272


In [8]:
submission.to_csv("../submissions/c4_en_b0_hv+flip_continued2.csv", index=False)